In [1]:
import numpy as np
import pandas as pd
from bokeh.layouts import layout
from bokeh.layouts import widgetbox
from bokeh.embed import file_html
from bokeh.io import show
from bokeh.io import output_notebook 
from bokeh.models import Text
from bokeh.models import Plot
from bokeh.models import Slider
from bokeh.models import Circle
from bokeh.models import Range1d
from bokeh.models import CustomJS
from bokeh.models import HoverTool
from bokeh.models import LinearAxis
from bokeh.models import ColumnDataSource
from bokeh.models import SingleIntervalTicker
from bokeh.palettes import Spectral6

In [2]:
import bokeh.sampledata

In [3]:
bokeh.sampledata.download()

Creating /home/ml/.bokeh directory
Creating /home/ml/.bokeh/data directory
Using data directory: /home/ml/.bokeh/data
Downloading: CGM.csv (1589982 bytes)
   1589982 [100.00%]
Downloading: US_Counties.zip (3182088 bytes)
   3182088 [100.00%]
Unpacking: US_Counties.csv
Downloading: us_cities.json (713565 bytes)
    713565 [100.00%]
Downloading: unemployment09.csv (253301 bytes)
    253301 [100.00%]
Downloading: AAPL.csv (166698 bytes)
    166698 [100.00%]
Downloading: FB.csv (9706 bytes)
      9706 [100.00%]
Downloading: GOOG.csv (113894 bytes)
    113894 [100.00%]
Downloading: IBM.csv (165625 bytes)
    165625 [100.00%]
Downloading: MSFT.csv (161614 bytes)
    161614 [100.00%]
Downloading: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip (5148539 bytes)
   5148539 [100.00%]
Unpacking: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.csv
Downloading: gapminder_fertility.csv (64346 bytes)
     64346 [100.00%]
Downloading: gapminder_population.csv (94509 bytes)
     94509 [100.00%]
Downloading: gapmind

In [3]:
def process_data():
    from bokeh.sampledata.gapminder import fertility, life_expectancy, population, regions
    # Make the column names ints not strings for handling
    columns = list(fertility.columns)
    years = list(range(int(columns[0]), int(columns[-1])))
    rename_dict = dict(zip(columns, years))

    fertility = fertility.rename(columns=rename_dict)
    life_expectancy = life_expectancy.rename(columns=rename_dict)
    population = population.rename(columns=rename_dict)
    regions = regions.rename(columns=rename_dict)

    # Turn population into bubble sizes. Use min_size and factor to tweak.
    scale_factor = 200
    population_size = np.sqrt(population / np.pi) / scale_factor
    min_size = 3
    population_size = population_size.where(population_size >= min_size).fillna(min_size)

    # Use pandas categories and categorize & color the regions
    regions.Group = regions.Group.astype('category')
    regions_list = list(regions.Group.cat.categories)

    def get_color(r):
        return Spectral6[regions_list.index(r.Group)]
    regions['region_color'] = regions.apply(get_color, axis=1)

    return fertility, life_expectancy, population_size, regions, years, regions_list

In [4]:
fertility_df, life_expectancy_df, population_df_size, regions_df, years, regions = process_data()
sources = {}
region_color = regions_df['region_color']
region_color.name = 'region_color'

In [13]:
for year in years:
    fertility = fertility_df[year]
    fertility.name = 'fertility'
    life = life_expectancy_df[year]
    life.name = 'life' 
    population = population_df_size[year]
    population.name = 'population' 
    new_df = pd.concat([fertility, life, population, region_color], axis=1)
    sources['_' + str(year)] = ColumnDataSource(new_df)

/home/ml/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [15]:
new_df.shape

(244, 4)

In [16]:
dictionary_of_sources = dict(zip([x for x in years], ['_%s' % x for x in years]))

In [17]:
dictionary_of_sources

{1964: '_1964',
 1965: '_1965',
 1966: '_1966',
 1967: '_1967',
 1968: '_1968',
 1969: '_1969',
 1970: '_1970',
 1971: '_1971',
 1972: '_1972',
 1973: '_1973',
 1974: '_1974',
 1975: '_1975',
 1976: '_1976',
 1977: '_1977',
 1978: '_1978',
 1979: '_1979',
 1980: '_1980',
 1981: '_1981',
 1982: '_1982',
 1983: '_1983',
 1984: '_1984',
 1985: '_1985',
 1986: '_1986',
 1987: '_1987',
 1988: '_1988',
 1989: '_1989',
 1990: '_1990',
 1991: '_1991',
 1992: '_1992',
 1993: '_1993',
 1994: '_1994',
 1995: '_1995',
 1996: '_1996',
 1997: '_1997',
 1998: '_1998',
 1999: '_1999',
 2000: '_2000',
 2001: '_2001',
 2002: '_2002',
 2003: '_2003',
 2004: '_2004',
 2005: '_2005',
 2006: '_2006',
 2007: '_2007',
 2008: '_2008',
 2009: '_2009',
 2010: '_2010',
 2011: '_2011',
 2012: '_2012'}

In [18]:
js_source_array = str(dictionary_of_sources).replace("'", "")
js_source_array

'{1964: _1964, 1965: _1965, 1966: _1966, 1967: _1967, 1968: _1968, 1969: _1969, 1970: _1970, 1971: _1971, 1972: _1972, 1973: _1973, 1974: _1974, 1975: _1975, 1976: _1976, 1977: _1977, 1978: _1978, 1979: _1979, 1980: _1980, 1981: _1981, 1982: _1982, 1983: _1983, 1984: _1984, 1985: _1985, 1986: _1986, 1987: _1987, 1988: _1988, 1989: _1989, 1990: _1990, 1991: _1991, 1992: _1992, 1993: _1993, 1994: _1994, 1995: _1995, 1996: _1996, 1997: _1997, 1998: _1998, 1999: _1999, 2000: _2000, 2001: _2001, 2002: _2002, 2003: _2003, 2004: _2004, 2005: _2005, 2006: _2006, 2007: _2007, 2008: _2008, 2009: _2009, 2010: _2010, 2011: _2011, 2012: _2012}'

In [32]:
xdr = Range1d(1, 9)
ydr = Range1d(20, 100)
plot = Plot(
    x_range=xdr,
    y_range=ydr,
    plot_width=800,
    plot_height=400,
    outline_line_color=None,
    toolbar_location=None, 
    min_border=20,)

In [33]:
show(plot)

In [34]:
AXIS_FORMATS = dict(
    minor_tick_in=None,
    minor_tick_out=None,
    major_tick_in=None,
    major_label_text_font_size="10pt",
    major_label_text_font_style="normal",
    axis_label_text_font_size="10pt",

    axis_line_color='#AAAAAA',
    major_tick_line_color='#AAAAAA',
    major_label_text_color='#666666',

    major_tick_line_cap="round",
    axis_line_cap="round",
    axis_line_width=1,
    major_tick_line_width=1,
)

xaxis = LinearAxis(ticker=SingleIntervalTicker(interval=1), axis_label="Children per woman (total fertility)", **AXIS_FORMATS)
yaxis = LinearAxis(ticker=SingleIntervalTicker(interval=20), axis_label="Life expectancy at birth (years)", **AXIS_FORMATS)   
plot.add_layout(xaxis, 'below')
plot.add_layout(yaxis, 'left')

In [35]:
show(plot)

In [36]:
text_source = ColumnDataSource({'year': ['%s' % years[0]]})
text = Text(x=35, y=35, text='year', text_font_size='150pt', text_color='red')
plot.add_glyph(text_source, text)

bokeh.models.renderers.GlyphRenderer(
    id='a502c76c-9c49-4eb8-9339-b21aab41502a',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='017768e4-a3f2-4a17-8006-4752fdfe4a49',
        callback=None,
        data={'year': ['1964']},
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        selected=bokeh.models.selections.Selection(
            id='3b743a08-cb5f-4e70-9e1c-cef063b549bb',
            indices=[],
            js_event_callbacks={},
            js_property_callbacks={},
            line_indices=[],
            multiline_indices={},
            name=None,
            subscribed_events=[],
            tags=[]),
        selection_policy=bokeh.models.selections.UnionRenderers(
            id='b6ca1531-34df-4ffc-b3ba-a88120f37f3a',
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    glyph=bokeh.models.glyphs.Text(
        id='00d61619-521f-49bb-a68f-7ba0ef10fa28',
        angle={'value': 0, 'units': 'rad'},
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        subscribed_events=[],
        tags=[],
        text={'field': 'year'},
        text_align='left',
        text_alpha={'value': 1.0},
        text_baseline='bottom',
        text_color={'value': 'red'},
        text_font='helvetica',
        text_font_size={'value': '150pt'},
        text_font_style='normal',
        text_line_height=1.2,
        x={'value': 35},
        x_offset={'value': 0},
        y={'value': 35},
        y_offset={'value': 0}),
    hover_glyph=None,
    js_event_callbacks={},
    js_property_callbacks={},
    level='glyph',
    muted=False,
    muted_glyph=None,
    name=None,
    nonselection_glyph='auto',
    selection_glyph='auto',
    subscribed_events=[],
    tags=[],
    view=bokeh.models.sources.CDSView(
        id='af763ddb-df09-436a-b62e-33182c100508',
        filters=[],
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        source=bokeh.models.sources.ColumnDataSource(
            id='017768e4-a3f2-4a17-8006-4752fdfe4a49',
            callback=None,
            data={'year': ['1964']},
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            selected=bokeh.models.selections.Selection(
                id='3b743a08-cb5f-4e70-9e1c-cef063b549bb',
                indices=[],
                js_event_callbacks={},
                js_property_callbacks={},
                line_indices=[],
                multiline_indices={},
                name=None,
                subscribed_events=[],
                tags=[]),
            selection_policy=bokeh.models.selections.UnionRenderers(
                id='b6ca1531-34df-4ffc-b3ba-a88120f37f3a',
                js_event_callbacks={},
                js_property_callbacks={},
                name=None,
                subscribed_events=[],
                tags=[]),
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    visible=True,
    x_range_name='default',
    y_range_name='default')

In [37]:
show(plot)

In [38]:
text_source.data

{'year': ['1964']}

In [39]:
# Add the circle
renderer_source = sources['_%s' % years[0]]
circle_glyph = Circle(
    x='fertility', y='life', size='population',
    fill_color='region_color', fill_alpha=0.8, 
    line_color='#7c7e71', line_width=0.5, line_alpha=0.5)

circle_renderer = plot.add_glyph(renderer_source, circle_glyph)

In [45]:
# Add the hover (only against the circle and not other plot elements)
tooltips = "@index"
plot.add_tools(HoverTool(tooltips=tooltips, renderers=[circle_renderer]))

In [42]:
show(plot)

In [48]:
text_x = 7
text_y = 95
for i, region in enumerate(regions):
    plot.add_glyph(Text(x=text_x, y=text_y, text=[region], text_font_size='10pt', text_color='#666666'))
    plot.add_glyph(Circle(x=text_x - 0.1, y=text_y + 2, fill_color=Spectral6[i], size=10, line_color=None, fill_alpha=0.8))
    text_y = text_y - 5

In [49]:
show(plot)

In [50]:
# Add the slider
code = """
    var year = slider.value,
        sources = %s,
        new_source_data = sources[year].data;
    renderer_source.data = new_source_data;
    text_source.data = {'year': [String(year)]};
""" % js_source_array

callback = CustomJS(args=sources, code=code)
slider = Slider(start=years[0], end=years[-1], value=1, step=1, title="Year", callback=callback)
callback.args["renderer_source"] = renderer_source
callback.args["slider"] = slider
callback.args["text_source"] = text_source

In [51]:
show(widgetbox(slider))

In [52]:
show(layout([[plot], [slider]], sizing_mode='scale_width'))

In [54]:
code

"\n    var year = slider.value,\n        sources = {1964: _1964, 1965: _1965, 1966: _1966, 1967: _1967, 1968: _1968, 1969: _1969, 1970: _1970, 1971: _1971, 1972: _1972, 1973: _1973, 1974: _1974, 1975: _1975, 1976: _1976, 1977: _1977, 1978: _1978, 1979: _1979, 1980: _1980, 1981: _1981, 1982: _1982, 1983: _1983, 1984: _1984, 1985: _1985, 1986: _1986, 1987: _1987, 1988: _1988, 1989: _1989, 1990: _1990, 1991: _1991, 1992: _1992, 1993: _1993, 1994: _1994, 1995: _1995, 1996: _1996, 1997: _1997, 1998: _1998, 1999: _1999, 2000: _2000, 2001: _2001, 2002: _2002, 2003: _2003, 2004: _2004, 2005: _2005, 2006: _2006, 2007: _2007, 2008: _2008, 2009: _2009, 2010: _2010, 2011: _2011, 2012: _2012},\n        new_source_data = sources[year].data;\n    renderer_source.data = new_source_data;\n    text_source.data = {'year': [String(year)]};\n"

In [55]:
help(CustomJS)

Help on class CustomJS in module bokeh.models.callbacks:

class CustomJS(Callback)
 |  CustomJS(**kwargs)
 |  
 |  Execute a JavaScript function.
 |  
 |  .. warning::
 |      The explicit purpose of this Bokeh Model is to embed *raw JavaScript
 |      code* for a browser to execute. If any part of the code is derived
 |      from untrusted user inputs, then you must take appropriate care to
 |      sanitize the user input prior to passing to Bokeh.
 |  
 |  Method resolution order:
 |      CustomJS
 |      Callback
 |      bokeh.model.Model
 |      bokeh.core.has_props.HasProps
 |      bokeh.util.callback_manager.PropertyCallbackManager
 |      bokeh.util.callback_manager.EventCallbackManager
 |      builtins.object
 |  
 |  Class methods defined here:
 |  
 |  from_coffeescript(code, args={}) from bokeh.model.MetaModel
 |      Create a ``CustomJS`` instance from CoffeeScript code.
 |  
 |  from_py_func(func) from bokeh.model.MetaModel
 |      Create a CustomJS instance from a Python 